In [1]:
# Some path wizardry to make python acknowledge relative paths, just ignore this part...
import sys; sys.path.append('..')
import networkx as nx
import pandas as pd
from ipysigma import Sigma
from pelote import to_nodes_dataframe, to_bipartite_graph

In [2]:
df = pd.read_csv('./data/bipartite.csv')
df

,account,url,weight
0,0,0,1
1,1,0,7
2,2,0,38
3,3,0,23
4,4,0,27
...,...,...,...
9350,347,793,25
9351,204,793,26
9352,1176,794,16
9353,308,795,7


In [3]:
g = to_bipartite_graph(df, 'account', 'url', first_part_data=lambda x: {'span': x['weight']})

In [4]:
to_nodes_dataframe(g, node_key_col=None)

,part,label,span
0,account,0,1.0
1,url,0,NaN
2,account,1,7.0
3,account,2,38.0
4,account,3,23.0
...,...,...,...
1995,url,792,NaN
1996,url,793,NaN
1997,url,794,NaN
1998,url,795,NaN
